In [1]:
%load_ext autoreload
%autoreload 2

from meteo_lib import PROJECT  # または: from meteo_lib.paths import PROJECT
#out_dir  = PROJECT / "outputs"

from meteo_lib.meteo_lib import getBlock, get_HourlyData, get_10minData
from meteo_lib.my_path_utils import DATA, OUT

import pandas as pd
import math
import numpy as np
from datetime import datetime, timedelta, date, time
from metpy.units import units
from metpy.calc import dewpoint_from_relative_humidity
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams["font.family"] = "IPAexGothic"   # 例：Noto Sans CJK JP でも可
mpl.rcParams["axes.unicode_minus"] = False    # －が豆腐になるのを回避

import matplotlib.dates as mdates
import matplotlib.gridspec as gridspec
from matplotlib.ticker import FuncFormatter, MultipleLocator
import os
from pathlib import Path
import calendar
from IPython.display import display 

In [2]:
def get_Webdata(prec, block, y_start, y_end, m_start, m_end, outdata, use_Flag):
    print(use_Flag)
    dfs = []
    for y in range(y_start, y_end+1):
        for m in range(m_start, m_end+1):
            days = calendar.monthrange(y, m)[1]
            for d in range(1, days+1):
                df_daily = get_HourlyData(prec, block, y, m, d)
                dfs.append(df_daily)
        print(y)
    df = pd.concat(dfs, axis=0, ignore_index=False)
    display(df)

    df.sort_index(inplace=True)
    df.to_csv(outdata)
    return df

In [29]:
dtype_map = {
    '現地気圧': 'float32',
    '海面気圧': 'float32',
    '降水量': 'float32',
    '気温': 'float32',
    '露点温度': 'float32',
    '蒸気圧': 'float32',
    '湿度': 'float32',
    '風速': 'float32',
    '風向': 'string',
    '日照': 'float32',
    '全天日射量': 'float32',
    '降雪': 'float32',
    '積雪': 'float32',
    '天気': 'string',
    '雲量': 'string',
    '視程': 'float32'
}
pref, station = '秋田県', '秋田'
info = getBlock(pref, station)
prec, block, st_name = info['prec'], info['block'], info["Name_2"]

y_start, y_end = 1961,2024
m_start, m_end = 1, 12

indata = DATA / f"Hourly_{station}_{y_start}-{y_end}.csv"
print(indata)

if indata.exists():
    print("from Local")
    df = pd.read_csv(
        indata,
        dtype=dtype_map,
        encoding='utf-8')
else:
    print("from Web")
    df = get_Webdata(prec, block, y_start, y_end, m_start, m_end, indata, True)

flag_cols = df.filter(regex=r'_flag$').columns.tolist()
print(flag_cols)
df = df.drop(columns=flag_cols, errors='ignore')

df['日時'] = pd.to_datetime(df['日時'])
print(df.dtypes)
df

/Users/takumi/Library/Mobile Documents/com~apple~CloudDocs/Projects/JMA-data/data/Hourly_秋田_1961-2024.csv
from Web
True
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024


,現地気圧,海面気圧,降水量,気温,露点温度,蒸気圧,湿度,風速,風向,日照,...,天気,雲量,視程,風速_flag,降水量_flag,全天日射量_flag,露点温度_flag,蒸気圧_flag,日照_flag,気温_flag
日時,,,,,,,,,,,,,,,,,,,,,
1961-01-01 01:00:00,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,,NaN,...,,,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1961-01-01 02:00:00,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,,NaN,...,,,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1961-01-01 03:00:00,1006.000000,1007.200012,0.0,-2.1,NaN,4.9,94.0,2.0,北,NaN,...,,10,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1961-01-01 04:00:00,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,,NaN,...,,,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1961-01-01 05:00:00,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,,NaN,...,,,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-31 20:00:00,1003.500000,1006.200012,2.5,1.3,-0.1,6.0,90.0,7.4,北北西,NaN,...,みぞれ,,2.34,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2024-12-31 21:00:00,1004.299988,1007.000000,2.0,1.4,0.3,6.2,92.0,6.8,北北西,NaN,...,みぞれ,,7.18,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2024-12-31 22:00:00,1004.700012,1007.400024,1.0,1.0,0.0,6.1,93.0,4.4,北,NaN,...,みぞれ,,20.00,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


['風速_flag', '降水量_flag', '全天日射量_flag', '露点温度_flag', '蒸気圧_flag', '日照_flag', '気温_flag']


KeyError: '日時'

In [ ]:
#df['日時'] = pd.to_datetime(df['日時'], errors='coerce')  # まず確実にdatetime化
df = df.set_index('日時').sort_index()

out_month = (
    df.groupby([pd.Grouper(freq='ME')])['気温']
      .agg(['mean', 'max', 'min', 'median', 'count'])
)
print(out_month)

In [ ]:
def monthly_climatology_mom(
    df, dt_col='日時', value_col='mean', *,
    start_year, end_year, wrap=False
):
    x = df.copy()
    x[dt_col] = pd.to_datetime(x[dt_col])

    # 1) 期間を切る
    m = (x.loc[x[dt_col].dt.year.between(start_year, end_year)]
           .resample('ME', on=dt_col)[value_col]        # 月末基準で月平均
           .mean()
           .to_frame('月平均'))

    # 2) 月番号で平年化（12値）
    m['月'] = m.index.month
    clim = (m.groupby('月')['月平均']
              .mean()
              .rename('基準月平均')           # 1991–2020の“平年”月平均
              .to_frame())

    # 3) 前月差（Jan は NaN。wrap=True なら Dec→Jan の差も計算）
    clim['前月差'] = clim['基準月平均'].diff()
    if wrap:
        # 12月→1月も含めた循環差
        prev = clim['基準月平均'].shift(1)
        prev.iloc[0] = clim['基準月平均'].iloc[-1]
        clim['前月差_cyclic'] = clim['基準月平均'] - prev

    clim = clim.reset_index()  # 列: 月, 基準月平均, 前月差(, 前月差_cyclic)
    return clim

# 使い方
diff_month = pd.DataFrame(index=range(1,13))
out_month['日時'] = out_month.index
#print(out_month)
ys = 1961
ye = 1991
#diff_month = []
for y in range(ys, ye+1, 10):
    clim_df = monthly_climatology_mom(out_month, dt_col='日時', 
          value_col='mean', start_year=y, end_year=y+11, wrap=True)
    if isinstance(clim_df, list) or isinstance(clim_df, tuple):
        clime_df = clim_df[0]
        
#    colname = f'{y}_diff_month'
    colname = f'{y}_{y+29}'
    s = clim_df.set_index('月')['前月差']
#    print('out_type: ',type(out))
    diff_month[colname] = s.reindex(diff_month.index).to_numpy()
#    print(y, out)

diff_month

In [ ]:
fig, ax = plt.subplots(figsize=(10,7))
for y in  range(ys, ye+1, 10):
    colname = f'{y}_{y+29}'
    ax.bar(diff_month.index, diff_month[colname], label=y)
ax.set_xlim(8, 11)
ax.legend()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
mpl.rcParams["font.family"] = "IPAexGothic"   # 例：Noto Sans CJK JP でも可
mpl.rcParams["axes.unicode_minus"] = False    # －が豆腐になるのを回避

cols = [f"{y}_{y+29}" for y in range(ys, ye+1, 10) 
        if f"{y}_{y+29}" in diff_month.columns]

ax = diff_month.loc[8:11, cols].plot(
    kind='bar', figsize=(10,7), rot=0)  # ← 横並びがデフォルト
ax.set_xlabel('月', fontsize=14)
ax.set_ylabel('前月差（℃）', fontsize=14)
# 目盛ラベルのフォントサイズ
ax.tick_params(axis='x', labelsize=14)
ax.tick_params(axis='y', labelsize=14)

ax.legend(title='平年値')
ax.axhline(0, linewidth=1)
plt.title(f'前月との差（平均気温、{station}）', fontsize=16)
plt.tight_layout()
plt.savefig(OUT / f'前月差_平均気温_平年値_{station}.png')
plt.show()
